# Traveling Salesman Problem (TSP) integrated with Google Maps Platform

### Libraries

In [1]:
import googlemaps
import gurobipy as gp
from gurobipy import GRB
import folium
from googlemaps.convert import decode_polyline

### Create Matrix of Distances

In [2]:
def get_distance_matrix(addresses, api_key):
    """
    Uses the Google Distance Matrix API to build a 2D list of driving
    distances (in meters) between all pairs of addresses.
    """
    gmaps = googlemaps.Client(key=api_key)

    result = gmaps.distance_matrix(
        origins=addresses,
        destinations=addresses,
        mode='driving',     # 'driving', 'walking', 'bicycling', or 'transit'
        units='metric'
    )
    n = len(addresses)
    dist_matrix = [[0.0]*n for _ in range(n)]

    for i in range(n):
        for j in range(n):
            element = result['rows'][i]['elements'][j]
            if element['status'] == 'OK':
                # Here we pick up distance in meters
                dist_matrix[i][j] = float(element['distance']['value'])
            else:
                dist_matrix[i][j] = float('inf')  # If no route is found
    return dist_matrix

### TSP Model

In [3]:
def solve_tsp(dist_matrix):
    """
    Solves TSP using Gurobi’s MTZ (Miller–Tucker–Zemlin) formulation.
    Returns (route, obj_val) where:
      - route is a list of nodes in the order of visitation
      - obj_val is the total distance (meters)
    """
    n = len(dist_matrix)
    m = gp.Model("TSP")

    # x[i, j] = 1 if route goes from i to j
    x = {}
    for i in range(n):
        for j in range(n):
            if i != j:
                x[(i, j)] = m.addVar(vtype=GRB.BINARY, name=f"x_{i}_{j}")
            else:
                # disallow i->i
                x[(i, j)] = m.addVar(vtype=GRB.BINARY, lb=0, ub=0, name=f"x_{i}_{j}_self")

    # u[i] for the MTZ constraints
    u = {}
    for i in range(n):
        u[i] = m.addVar(lb=0, ub=n-1, vtype=GRB.CONTINUOUS, name=f"u_{i}")

    m.update()

    # Objective: minimize total distance
    m.setObjective(
        gp.quicksum(dist_matrix[i][j] * x[(i, j)] for i in range(n) for j in range(n)),
        GRB.MINIMIZE
    )

    # Each location has exactly one incoming edge
    for j in range(n):
        m.addConstr(gp.quicksum(x[(i, j)] for i in range(n)) == 1, name=f"in_{j}")

    # Each location has exactly one outgoing edge
    for i in range(n):
        m.addConstr(gp.quicksum(x[(i, j)] for j in range(n)) == 1, name=f"out_{i}")

    # MTZ constraints to eliminate subtours
    for i in range(1, n):
        for j in range(1, n):
            if i != j:
                m.addConstr(u[i] - u[j] + n*x[(i, j)] <= n-1, name=f"mtz_{i}_{j}")

    m.optimize()

    if m.status == GRB.OPTIMAL:
        obj_val = m.objVal
        # Reconstruct route
        route = [0]
        visited = 1
        current = 0
        while visited < n:
            for j in range(n):
                if x[(current, j)].X > 0.5:
                    route.append(j)
                    current = j
                    break
            visited += 1
        return route, obj_val
    else:
        print("No optimal solution found.")
        return None, None

### Latitude/longitude

In [4]:
def geocode_addresses(addresses, api_key):
    """
    Get latitude/longitude for each address using Google Geocoding API.
    Returns list of (lat, lng) in the same order as 'addresses'.
    """
    gmaps = googlemaps.Client(key=api_key)
    coords = []
    for addr in addresses:
        result = gmaps.geocode(addr)
        if result:
            loc = result[0]["geometry"]["location"]
            coords.append((loc["lat"], loc["lng"]))
        else:
            coords.append((0.0, 0.0))
    return coords

### Plot

In [5]:
def plot_route_on_map_with_directions(route, addresses, coords, api_key, output_html="tsp_route.html"):
    """
    Plot the TSP route as an actual road path by
    calling Google Directions API for each pair of consecutive stops.
    Saves the map to an HTML file for interactive viewing.
    """
    gmaps = googlemaps.Client(key=api_key)

    # Start map near the first location
    start_lat, start_lng = coords[route[0]]
    fmap = folium.Map(location=[start_lat, start_lng], zoom_start=13)

    # Add markers in the visit order
    for order, node_idx in enumerate(route):
        lat, lng = coords[node_idx]
        folium.Marker(
            [lat, lng],
            popup=f"Stop {order+1}: {addresses[node_idx]}",
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(fmap)

    # Draw polylines via real road paths
    for i in range(len(route) - 1):
        origin_idx = route[i]
        dest_idx = route[i+1]

        origin_lat, origin_lng = coords[origin_idx]
        dest_lat, dest_lng = coords[dest_idx]

        # Query Directions API for the road path
        directions_result = gmaps.directions(
            (origin_lat, origin_lng),
            (dest_lat, dest_lng),
            mode="driving"
        )
        if directions_result:
            # Extract the overview polyline and decode
            polyline_str = directions_result[0]['overview_polyline']['points']
            polyline_points = decode_polyline(polyline_str)

            folium.PolyLine(
                locations=[(pt['lat'], pt['lng']) for pt in polyline_points],
                color='red',
                weight=3,
                opacity=1
            ).add_to(fmap)
        else:
            print(f"No driving directions found from {addresses[origin_idx]} to {addresses[dest_idx]}.")

    fmap.save(output_html)
    print(f"Map with road paths saved to {output_html}.")

### Choose Locations

In [6]:
# 8 locations around Midtown Atlanta, with starting point at Georgia Tech
addresses = [
    "Georgia Tech, 225 North Ave NW, Atlanta, GA 30332",
    "Fox Theatre, 660 Peachtree St NE, Atlanta, GA 30308",
    "Atlanta Botanical Garden, 1345 Piedmont Ave NE, Atlanta, GA 30309",
    "Ponce City Market, 675 Ponce De Leon Ave NE, Atlanta, GA 30308",
    "Coca-Cola Headquarters, 1 Coca Cola Plz NW, Atlanta, GA 30313",
    "Mercedes-Benz Stadium, 1 AMB Dr NW, Atlanta, GA 30313",
    "Georgia Aquarium, 225 Baker St NW, Atlanta, GA 30313",
    "World of Coca-Cola, 121 Baker St NW, Atlanta, GA 30313"
]

### API Key

In [7]:
# Replace with your valid Google Cloud API key
api_key = "YOUR_API"

### Solve the Model

In [8]:
# 1. Build TSP distance matrix
dist_matrix = get_distance_matrix(addresses, api_key)

# 2. Solve TSP
route, obj_val = solve_tsp(dist_matrix)
if route is None:
    print("No feasible TSP route found.")

print("Optimal TSP route (indices):", route)
print("Total distance (meters):", obj_val)

# 3. Geocode to get lat/lng
coords = geocode_addresses(addresses, api_key)

# 4. Plot the route with real roads
plot_route_on_map_with_directions(route, addresses, coords, api_key, output_html="tsp_route.html")

print("Done. Open 'tsp_route.html'.")


Set parameter Username
Academic license - for non-commercial use only - expires 2026-03-13
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (26100.2))

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 58 rows, 72 columns and 254 nonzeros
Model fingerprint: 0xa6ecad86
Variable types: 8 continuous, 64 integer (64 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [2e+02, 6e+03]
  Bounds range     [1e+00, 7e+00]
  RHS range        [1e+00, 7e+00]
Found heuristic solution: objective 19745.000000
Presolve removed 0 rows and 9 columns
Presolve time: 0.00s
Presolved: 58 rows, 63 columns, 448 nonzeros
Variable types: 7 continuous, 56 integer (56 binary)

Root relaxation: objective 1.219886e+04, 19 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |   